In [21]:
from boltons.pathutils import augpath
from PIL import Image
import matplotlib.pyplot as plt

In [35]:
def show(img):
    plt.figure(figsize=(8, 8))
    plt.axis("off")
    plt.imshow(img)
    plt.show()

In [42]:
def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    max_height = max(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * max_height / im.height), max_height),resample=resample)
                      for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, max_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                      for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

def get_concat_tile_resize(im_list_2d, resample=Image.BICUBIC):
    im_list_v = [get_concat_h_multi_resize(im_list_h, resample=resample) for im_list_h in im_list_2d]
    return get_concat_v_multi_resize(im_list_v, resample=resample)

In [46]:
image_dir = "../images"
lowpoly_dir = "../images/lowpoly"
lowpoly_ext = "png"
files = {
    "bird1.jpg": (1000, 0, 5000, 4000),
    "fox.jpg": (50, 0, 910, 860),
    "eminem.jpg": (0, 0, 400, 400),
}

images = []
for file, box in files.items():
    file = augpath(file, dpath=image_dir)
    image = Image.open(file)
    lowpoly_image = Image.open(augpath(file, dpath=lowpoly_dir, ext=".png")).resize(image.size)

    image_cropped = image.crop(box)
    lowpoly_image_cropped = lowpoly_image.crop(box)
    images.append(image_cropped)
    images.append(lowpoly_image_cropped)

#     show(image_cropped)
#     show(lowpoly_image_cropped)
# max_size = max([img.size for img in images])
# images = [img.resize(max_size) for img in images]
# for img in images:
#     show(img)

get_concat_tile_resize([
    [images[0], images[2], images[4]],
    [images[1], images[3], images[5]],
]).save("concat_original.jpg", quality=60, optimize=True, progressive=True)